In [17]:
import os
import json
import glob
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil
import random

# Train data generation
import collections
import csv
from pathlib import Path
from typing import List, Tuple, Any

import time
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import pickle

pd.set_option("display.max_columns", 100)

In [ ]:
SEED = 42

In [18]:
dir_path = "../input/indoorpkl/"
train_file_name = dir_path + "indoor_train.pkl"
test_file_name = dir_path + "indoor_test.pkl"

# Try loading it back in
with open(train_file_name, "rb") as file:
    df_train = pickle.load(file)

with open(test_file_name, "rb") as file:
    df_test = pickle.load(file)

In [19]:
display(df_train.head())
display(df_test.head())

,site_id,file_id,floor_converted,floor,ts,x,y,start_ts,diff_start_ts,acce_ts,acce_x,acce_y,acce_z,acce_acc,ahrs_ts,ahrs_x,ahrs_y,ahrs_z,ahrs_acc,magn_ts,magn_x,magn_y,magn_z,magn_acc,wifi_ts,wifi_ssid,wifi_bssid,wifi_rssi,wifi_freq,wifi_last_seen_ts,diff_acce_ts_start_ts,diff_ahrs_ts_start_ts,diff_magn_ts_start_ts,diff_wifi_ts_start_ts,site_id_le,file_id_le,floor_converted_le,wifi_ssid_le,wifi_bssid_le,ts_date,ts_day,ts_hour,ts_minute,wifi_last_seen_ts_date,wifi_last_seen_ts_day,wifi_last_seen_ts_hour,wifi_last_seen_ts_minute
0,5cd56c0ce2acfd2d33b6ab27,5d09a625bd54340008acddb9,-1,B1,1.560913e+12,14.283729,20.392578,1.560913e+12,0.0,1.560913e+12,-0.210693,-0.304062,9.943115,NaN,1.560913e+12,-0.012902,0.008711,-0.427844,NaN,1.560913e+12,-21.720000,17.76,-36.120000,NaN,1.560913e+12,bd56240b1064c9e8e62ec3b8b1825d1104c16dcc,51e058eb65d3e5b3838e8dba0f3006028d5fd864,-90.0,NaN,1.560913e+12,186.0,186.0,186.0,530.0,0,3,0,22,19,2019-06-19 03:02:49.585999872,2019-06-19,2019-06-19 03:00:00,2019-06-19 03:02:00,2019-06-19 03:02:43.913999872,2019-06-19,2019-06-19 03:00:00,2019-06-19 03:02:00
1,5cd56c0ce2acfd2d33b6ab27,5d09a625bd54340008acddb9,-1,B1,1.560913e+12,21.157534,30.024122,1.560913e+12,9886.0,1.560913e+12,-0.718262,-0.418991,10.347733,NaN,1.560913e+12,-0.006788,0.039279,-0.327645,NaN,1.560913e+12,-19.260000,18.90,-30.179998,NaN,1.560913e+12,bd56240b1064c9e8e62ec3b8b1825d1104c16dcc,51e058eb65d3e5b3838e8dba0f3006028d5fd864,-84.0,NaN,1.560913e+12,9895.0,9895.0,9895.0,9998.0,0,3,0,22,19,2019-06-19 03:02:59.472000000,2019-06-19,2019-06-19 03:00:00,2019-06-19 03:02:00,2019-06-19 03:02:52.752999936,2019-06-19,2019-06-19 03:00:00,2019-06-19 03:02:00
0,5cd56c0ce2acfd2d33b6ab27,5d09a625bd54340008acddb7,-1,B1,1.560913e+12,10.195710,21.657787,1.560913e+12,0.0,1.560913e+12,0.043091,0.275330,9.428360,NaN,1.560913e+12,0.000129,0.000146,-0.757510,NaN,1.560913e+12,-24.180000,-5.16,-26.640000,NaN,1.560913e+12,bd56240b1064c9e8e62ec3b8b1825d1104c16dcc,51e058eb65d3e5b3838e8dba0f3006028d5fd864,-87.0,NaN,1.560913e+12,192.0,192.0,192.0,502.0,0,2,0,22,19,2019-06-19 03:00:25.966000128,2019-06-19,2019-06-19 03:00:00,2019-06-19 03:00:00,2019-06-19 03:00:22.655000064,2019-06-19,2019-06-19 03:00:00,2019-06-19 03:00:00
1,5cd56c0ce2acfd2d33b6ab27,5d09a625bd54340008acddb7,-1,B1,1.560913e+12,14.283729,20.392578,1.560913e+12,5292.0,1.560913e+12,-0.541092,1.374268,10.240006,NaN,1.560913e+12,0.078497,-0.037847,-0.717117,NaN,1.560913e+12,-23.820000,-6.90,-26.880000,NaN,1.560913e+12,bd56240b1064c9e8e62ec3b8b1825d1104c16dcc,51e058eb65d3e5b3838e8dba0f3006028d5fd864,-76.0,NaN,1.560913e+12,5284.0,5284.0,5284.0,5531.0,0,2,0,22,19,2019-06-19 03:00:31.257999872,2019-06-19,2019-06-19 03:00:00,2019-06-19 03:00:00,2019-06-19 03:00:31.092000000,2019-06-19,2019-06-19 03:00:00,2019-06-19 03:00:00
2,5cd56c0ce2acfd2d33b6ab27,5d09a625bd54340008acddb7,-1,B1,1.560913e+12,21.089481,19.001072,1.560913e+12,13045.0,1.560913e+12,-0.131683,0.172379,10.309433,NaN,1.560913e+12,0.017186,-0.002997,-0.696129,NaN,1.560913e+12,-26.699999,1.86,-10.620000,NaN,1.560913e+12,bd56240b1064c9e8e62ec3b8b1825d1104c16dcc,51e058eb65d3e5b3838e8dba0f3006028d5fd864,-79.0,NaN,1.560913e+12,13051.0,13051.0,13051.0,13216.0,0,2,0,22,19,2019-06-19 03:00:39.011000064,2019-06-19,2019-06-19 03:00:00,2019-06-19 03:00:00,2019-06-19 03:00:33.140999936,2019-06-19,2019-06-19 03:00:00,2019-06-19 03:00:00


,site_id,file_id,floor_converted,floor,ts,x,y,start_ts,diff_start_ts,acce_ts,acce_x,acce_y,acce_z,acce_acc,ahrs_ts,ahrs_x,ahrs_y,ahrs_z,ahrs_acc,magn_ts,magn_x,magn_y,magn_z,magn_acc,wifi_ts,wifi_ssid,wifi_bssid,wifi_rssi,wifi_freq,wifi_last_seen_ts,diff_acce_ts_start_ts,diff_ahrs_ts_start_ts,diff_magn_ts_start_ts,diff_wifi_ts_start_ts,site_id_le,file_id_le,floor_converted_le,wifi_ssid_le,wifi_bssid_le,ts_date,ts_day,ts_hour,ts_minute,wifi_last_seen_ts_date,wifi_last_seen_ts_day,wifi_last_seen_ts_hour,wifi_last_seen_ts_minute
0,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,9.0,NaN,NaN,0.0,9.0,136.0,0.798813,4.300720,7.810059,NaN,136.0,0.247101,0.104201,0.474897,NaN,136.0,30.561829,-1.228333,-38.301086,NaN,2340.0,da39a3ee5e6b4b0d3255bfef95601890afd80709,eebf5db207eec2f3e041f92153d789270f346821,-45.0,NaN,1.578475e+12,136.0,136.0,136.0,2340.0,0,0,0,108,264,1970-01-01 00:00:00.009,1970-01-01,1970-01-01,1970-01-01,2020-01-08 09:09:04.726000128,2020-01-08,2020-01-08 09:00:00,2020-01-08 09:09:00
1,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,9017.0,NaN,NaN,0.0,9017.0,9012.0,-1.106979,4.056503,9.795456,NaN,9012.0,0.162119,0.185954,0.561409,NaN,9012.0,29.867554,-6.085205,-26.150513,NaN,9508.0,da39a3ee5e6b4b0d3255bfef95601890afd80709,1d1d62dcf72481cc9580fed3b724f0d27015aaf1,-43.0,NaN,1.578475e+12,9012.0,9012.0,9012.0,9508.0,0,0,0,108,35,1970-01-01 00:00:09.017,1970-01-01,1970-01-01,1970-01-01,2020-01-08 09:09:30.052000000,2020-01-08,2020-01-08 09:00:00,2020-01-08 09:09:00
2,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,15326.0,NaN,NaN,0.0,15326.0,15326.0,-0.572464,3.981689,7.082230,NaN,15326.0,0.225070,0.022647,-0.200452,NaN,15326.0,-6.207275,14.727783,-39.649963,NaN,14714.0,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,b26914599f6d9ba16b43975394e1eeb9d82f4bab,-41.0,NaN,1.578475e+12,15326.0,15326.0,15326.0,14714.0,0,0,0,79,208,1970-01-01 00:00:15.326,1970-01-01,1970-01-01,1970-01-01,2020-01-08 09:09:38.027000064,2020-01-08,2020-01-08 09:00:00,2020-01-08 09:09:00
3,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,18763.0,NaN,NaN,0.0,18763.0,18755.0,-0.751434,4.546112,10.231201,NaN,18755.0,0.225055,0.044806,-0.119175,NaN,18755.0,-1.350403,9.870911,-41.676330,NaN,19587.0,da39a3ee5e6b4b0d3255bfef95601890afd80709,de53ffe7e3c71c9ed5c845fa50e0521efa5f3685,-41.0,NaN,1.578475e+12,18755.0,18755.0,18755.0,19587.0,0,0,0,108,249,1970-01-01 00:00:18.763,1970-01-01,1970-01-01,1970-01-01,2020-01-08 09:09:42.716999936,2020-01-08,2020-01-08 09:00:00,2020-01-08 09:09:00
4,5a0546857ecc773753327266,046cfa46be49fc10834815c6,NaN,NaN,22328.0,NaN,NaN,0.0,22328.0,22326.0,-2.089798,4.224701,12.037628,NaN,22326.0,0.242105,0.053464,-0.008162,NaN,22326.0,-1.350403,5.014038,-30.876160,NaN,22074.0,da39a3ee5e6b4b0d3255bfef95601890afd80709,1d1d62dcf72481cc9580fed3b724f0d27015aaf1,-42.0,NaN,1.578475e+12,22326.0,22326.0,22326.0,22074.0,0,0,0,108,35,1970-01-01 00:00:22.328,1970-01-01,1970-01-01,1970-01-01,2020-01-08 09:09:34.860999936,2020-01-08,2020-01-08 09:00:00,2020-01-08 09:09:00


In [118]:
# metric
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat - x,2) + np.power(yhat-y,2)) + 15 * np.abs(fhat-f)
    return intermediate.sum() / xhat.shape[0]

In [135]:
# column settings
drop_cols = ["floor_converted", "floor", "x", "y", "floor_converted_le"]
categorical_features = ["site_id", "file_id", "wifi_ssid", "wifi_bssid"]
datetime_features = ["ts_date", "ts_day", "ts_hour", "ts_minute", \
                     "wifi_last_seen_ts_date", "wifi_last_seen_ts_day", \
                     "wifi_last_seen_ts_hour", "wifi_last_seen_ts_minute"]

# convert to category from object dtype
for col in categorical_features:
    df_train[col] = df_train[col].astype("category")
    
# convert to int from datetime64 dtype
for col in datetime_features:
    df_train[col] = df_train[col].astype(int)

# set features and labels
# print(df_train.info())

features = df_train.drop(columns=drop_cols)
target_x = df_train.iloc[:, 5]
target_y = df_train.iloc[:, 6]
target_f = df_train.iloc[:, 2]

targets = ["x", "y", "f"]
target_data = [target_x, target_y, target_f]

d = {}
for tgt, tgt_data in zip(targets, target_data):
    feat_train, feat_val, target_train, target_val = \
    train_test_split(features, tgt_data, test_size = 0.2, random_state = 42)
    d["feat_train_{}".format(tgt)] = feat_train
    d["feat_val_{}".format(tgt)] = feat_val
    d["target_train_{}".format(tgt)] = target_train
    d["target_val_{}".format(tgt)] = target_val
    d["train_{}".format(tgt)] = lgb.Dataset(data=feat_train, label=target_train, categorical_feature=categorical_features, free_raw_data=False).construct()
    d["val_{}".format(tgt)] = lgb.Dataset(data=feat_val, label=target_val, categorical_feature=categorical_features, free_raw_data=False).construct()
    print(d["feat_train_{}".format(tgt)].shape)
    print(d["feat_val_{}".format(tgt)].shape)
    print(d["target_train_{}".format(tgt)].shape)
    print(d["target_val_{}".format(tgt)].shape)
    print(d["train_{}".format(tgt)].get_data().shape)
    print(d["train_{}".format(tgt)].get_label().shape)
    print(d["val_{}".format(tgt)].get_data().shape)
    print(d["val_{}".format(tgt)].get_label().shape)

(428, 42)
(108, 42)
(428,)
(108,)
(428, 42)
(428,)
(108, 42)
(108,)
(428, 42)
(108, 42)
(428,)
(108,)
(428, 42)
(428,)
(108, 42)
(108,)
(428, 42)
(108, 42)
(428,)
(108,)
(428, 42)
(428,)
(108, 42)
(108,)


In [136]:
# lgb patams
lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000, # example had 50000
              'learning_rate': 0.1,
              'num_leaves': 90,
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

# train models
for tgt in targets:
    model = lgb.train(params=lgb_params,
                      train_set=d["train_{}".format(tgt)],
                      early_stopping_rounds=20,
                      valid_sets=d["val_{}".format(tgt)])
    d["model_{}".format(tgt)] = model
    d["pred_target_{}".format(tgt)] = model.predict(d["feat_val_{}".format(tgt)], 
                                                    num_iteration=model.best_iteration)

mse_x = mean_squared_error(d["target_val_x"], d["pred_target_x"])
mse_y = mean_squared_error(d["target_val_y"], d["pred_target_y"])
mse_f = mean_squared_error(d["target_val_f"], d["pred_target_f"])
rmse_x = np.sqrt(mse_x)
rmse_y = np.sqrt(mse_y)
rmse_f = np.sqrt(mse_f)
print("rmse_x:", rmse_x, "rmse_y:",rmse_y, "rmse_f:",rmse_f)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000475 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3706
[LightGBM] [Info] Number of data points in the train set: 428, number of used features: 38
[LightGBM] [Info] Start training from score 116.858280
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's rmse: 55.4332
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's rmse: 51.3066
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's rmse: 47.4879
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's rmse: 44.21
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's rmse: 41.5416
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	va

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[19]	valid_0's rmse: 17.6955
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[20]	valid_0's rmse: 17.6128
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[21]	valid_0's rmse: 17.3792
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[22]	valid_0's rmse: 17.1698
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[23]	valid_0's rmse: 17.0619
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[24]	valid_0's rmse: 16.8977
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[25]	valid_0's rmse: 16.8563
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[26]	valid_0's rmse: 16.6203
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[27]	valid_0's rmse: 16.5772
[LightGBM] [Warning] No further splits with positive gain, best 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[214]	valid_0's rmse: 13.6566
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[215]	valid_0's rmse: 13.6453
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[216]	valid_0's rmse: 13.6311
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[217]	valid_0's rmse: 13.6137
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[218]	valid_0's rmse: 13.6295
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[219]	valid_0's rmse: 13.6243
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[220]	valid_0's rmse: 13.6529
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[221]	valid_0's rmse: 13.6581
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[222]	valid_0's rmse: 13.6661
[LightGBM] [Warning] No further splits with positive ga

In [132]:
print(d.keys())

dict_keys(['feat_train_x', 'feat_val_x', 'target_train_x', 'target_val_x', 'train_x', 'val_x', 'feat_train_y', 'feat_val_y', 'target_train_y', 'target_val_y', 'train_y', 'val_y', 'feat_train_f', 'feat_val_f', 'target_train_f', 'target_val_f', 'train_f', 'val_f', 'model_x', 'pred_target_x', 'model_y', 'pred_target_y', 'model_f', 'pred_target_f'])


In [139]:
# print(len(d["target_val_x"]))
# print(d["target_val_x"].to_numpy())
# print(type(d["target_val_x"].to_numpy()))

# print(len(d["pred_target_x"]))
# print(d["pred_target_x"])
# print(type(d["pred_target_x"]))

# print(np.power(d["pred_target_x"] - d["target_val_x"].to_numpy(), 2))
# print(np.power(d["pred_target_y"] - d["target_val_y"].to_numpy(), 2))
a = np.sqrt(np.power(d["pred_target_x"] - d["target_val_x"].to_numpy(), 2) + np.power(d["pred_target_y"] - d["target_val_y"].to_numpy(), 2)) + 15 * np.abs(d["pred_target_f"] - d["target_val_f"].to_numpy())
print(a.sum() / d["pred_target_x"].shape[0])
print(d["pred_target_x"].shape[0])

comp_metric = comp_metric(d["pred_target_x"], d["pred_target_y"], d["pred_target_f"], 
                          d["target_val_x"].to_numpy(), d["target_val_y"].to_numpy(), d["target_val_f"].to_numpy())
print("comp metric: ", comp_metric)
# def comp_metric(xhat, yhat, fhat, x, y, f):
#     intermediate = np.sqrt(np.power(xhat - x,2) + np.power(yhat-y,2)) + 15 * np.abs(fhat-f)
#     return intermediate.sum()/xhat.shape[0]

24.78382305654363
108


TypeError: 'numpy.float64' object is not callable

In [ ]:
# # Load test data
# df_test_pred = df_test.drop(columns=drop_cols)

# # Add "site_path_timestamp" column to df_test
# # convert ts to unix and then convert it to string, and put it in a new column["ts_unix"]
# # join ["site_id", "file_id", "ts_unix"] and put it into a new column ["site_path_timestamp"]
# # or just add it when generating the data -> less steps -> less likely to make a mistake

# test_preds_x = model_x.predict(df_test_pred)
# test_preds_y = model_y.predict(df_test_pred)
# test_preds_f = model_f.predict(df_test_pred)

# # stack the results up with the pairing site_path_timestamp
# # load ssubm
# # test_preds = pd.concat([test_preds_x, test_preds_y, test_preds_f], axis=1)
# test_preds.columns = ssubm.columns
# test_preds.index = df_test_pred["site_path_timestamp"]
# test_preds["floor"] = test_preds["floor"].astype(int)
# predictions.append(test_preds)

In [ ]:
# generate prediction file 
# pd.concat(predictions)
# reindex the prediction df
# convert to csv for submission